In [1]:
# Decorators

def reciprocal(f):
    def rec(x):
        return 1 / f(x)
    return rec

@reciprocal
def times2(x):
    return 2 * x

times2(2)

0.25

In [2]:
# Decorator Generators

def a_over_f(a):
    def dec(f):
        def rec(x):
            return a / f(x)
        return rec
    return dec

@a_over_f(2)
def times2(x):
    return 2 * x

times2(2)

0.5

In [3]:
# Generators

def fibonacci(n):
    if n < 2:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

def fib(n):
    """Generate the fibonacci sequence up to n"""
    for i in range(n):
        yield fibonacci(i)
        
for i in fib(20):
    print(i, end=" ")

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 

In [4]:
# Slicing in reverse (just a test)

x = list(range(10))

x[5:2:-1]

[5, 4, 3]

In [5]:
# Orbitals and multiprocessing

import numpy as np
from multiprocessing import Pool

def remove_i(x, i):
    """Drops the ith element of an array."""
    shape = (x.shape[0]-1,) + x.shape[1:]
    y = np.empty(shape, dtype=float)
    y[:i] = x[:i]
    y[i:] = x[i+1:]
    return y

def a(i, x, G, m):
    """The acceleration of the ith mass."""
    x_i = x[i]
    x_j = remove_i(x, i)
    m_j = remove_i(m, i)
    diff = x_j - x_i
    mag3 = np.sum(diff**2, axis=1)**1.5
    result = G * np.sum(diff * (m_j / mag3)[:,np.newaxis], axis=0)
    return result

def initial_cond(N, D):
    """Generates initial conditions for N unity masses at rest 
    starting at random positions in D-dimensional space.
    """
    x0 = np.random.rand(N, D)
    v0 = np.zeros((N, D), dtype=float)
    m = np.ones(N, dtype=float)
    return x0, v0, m 

def timestep_i(args):
    """Computes the next position and velocity for the ith mass."""
    i, x0, v0, G, m, dt = args
    a_i0 = a(i, x0, G, m)
    v_i1 = a_i0 * dt + v0[i]
    x_i1 = a_i0 * dt**2 + v0[i] * dt + x0[i]
    return i, x_i1, v_i1

def timestep(x0, v0, G, m, dt, pool):
    """Computes the next position and velocity for all masses given 
    initial conditions and a time step size.
    """
    N = len(x0)
    tasks = [(i, x0, v0, G, m, dt) for i in range(N)]
    results = pool.map(timestep_i, tasks)
    x1 = np.empty(x0.shape, dtype=float)
    v1 = np.empty(v0.shape, dtype=float)
    for i, x_i1, v_i1 in results:
        x1[i] = x_i1
        v1[i] = v_i1
    return x1, v1

def simulate(P, N, D, S, G, dt):
    x0, v0, m = initial_cond(N, D)
    pool = Pool(P)
    for s in range(S):
        x1, v1 = timestep(x0, v0, G, m, dt, pool)
        x0, v0 = x1, v1
    pool.close()

In [6]:
import time
Ps = [1, 2, 4, 8, 16]
runtimes = {}
for P in Ps:
    start = time.time()
    simulate(P, 256, 3, 300, 1.0, 1e-3)
    stop = time.time()
    runtimes[P] = stop - start
    
runtimes

{1: 10.433343887329102,
 2: 5.022290945053101,
 4: 3.2955949306488037,
 8: 3.5057249069213867,
 16: 4.491585969924927}

In [7]:
%%capture

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def sim(P, N, D, S, G, dt):
    x0 = np.array([[1, 1], [-1, 1], [-1, -1], [1, -1]])
    v0 = np.array([[-0.02, -0.04], [0, 0], [0, 0], [0, 0.01]])
    m = np.ones(N, dtype=float)
    pool = Pool(P)
    results = [(x0, v0)]
    for s in range(S):
        x0, v0 = timestep(x0, v0, G, m, dt, pool)
        results.append((x0, v0))
    pool.close()
    return results

s = sim(8, 4, 2, 100, 0.002, 1)

fig = plt.figure(figsize=(5, 5))
plt.axis([-2, 2, -2, 2])
p1, = plt.plot([], [], 'o')
p2, = plt.plot([], [], 'o')
p3, = plt.plot([], [], 'o')
p4, = plt.plot([], [], 'o')

def draw(i):
    x, _ = s[i]
    p1.set_data(x[0, 0], x[0, 1])
    p2.set_data(x[1, 0], x[1, 1])
    p3.set_data(x[2, 0], x[2, 1])
    p4.set_data(x[3, 0], x[3, 1])
    return p1, p2, p3, p4

anim = FuncAnimation(fig, draw, 100, interval=100, blit=True)

In [8]:
from IPython.display import HTML
HTML(anim.to_html5_video())